# Desafio Engenheiro de Dados

**Criação do contexto:**

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()


**Importando os arquivos de log em duas RDDs:**

In [2]:
rddJuly = sc.textFile("access_log_Jul95").cache()
rddAugust = sc.textFile("access_log_Aug95").cache()


**Verificando se as RDDs foram criadas:**

In [3]:
type(rddJuly)


pyspark.rdd.RDD

In [4]:
type(rddAugust)

pyspark.rdd.RDD

**Imprimindo as dez primeiras linhas dos RDDs para testar se foi importado com sucesso:**

In [5]:
print(rddJuly.take(10))

['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245', 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985', '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085', 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0', '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179', 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0', 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0', '205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985', 'd104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985', '129.94.144.152 - - [01/Jul/1995:00:0

In [6]:
print(rddAugust.take(10))

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839', 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0', 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0', 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0', 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0', 'ix-esc-ca2-07.ix.netcom.com - - [01/Aug/1995:00:00:09 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713', 'uplherc.upl.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/WORLD-logosmall.gif HTTP/1.0" 304 0', 'slppp6.intermind.net - - [01/Aug/1995:00:00:10 -0400] "GET /history/skylab/skylab.html HTTP/1.0" 200 1687', 'piweba4y.prodigy.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/launchmedium.gif HTTP/1.0" 200 11853', 'slppp6.intermind.net - - [01/Aug/1995:00:00:11 -0400] "GET /history

# Questões


**Responda as seguintes questões devem ser desenvolvidas em Spark utilizando a sua linguagem de preferência.**

1. Número de hosts únicos.

In [7]:
JulyHosts = rddJuly.flatMap(lambda line: line.split(' ')[0]).distinct().count()

print('July Distinct Hosts: %s' % JulyHosts)

July Distinct Hosts: 55


In [8]:
AugustHosts = rddAugust.flatMap(lambda line: line.split(' ')[0]).distinct().count()

print('August Distinct Hosts: %s' % AugustHosts)

August Distinct Hosts: 53


2. O total de erros 404.

In [9]:
def check_404(l):
    try:
        if l.split(' ')[-2] == '404':
            return True
    except:
        return False

Error404July = dict(rddJuly.filter(check_404)
                           .filter(lambda line: '404' in line)
                           .map(lambda x: (x.split( )[-2],1))
                           .reduceByKey(lambda k,v : k+v).collect())

print('Total July Error 404:', Error404July['404'])

Total July Error 404: 10845


In [10]:
#Primeira Solução
Error404August = rddAugust.map(lambda x: x.split( )[-2]).filter(lambda line: "404" in line).countByValue()

print('Total August Error 404:', Error404August['404'])

Total August Error 404: 10056


In [11]:
#Segunda Solução, igual a anterior mas utilizando reduceByKey() ao invés de countByKey()
Error404August = dict(rddAugust.map(lambda x: (x.split( )[-2], 1))
                               .filter(lambda line: "404" in line)
                               .reduceByKey(lambda k,v : k+v).collect())

print('Total August Error 404:', Error404August['404'])

Total August Error 404: 10056


3. Os 5 URLs que mais causaram erro 404.

In [12]:
def check_404(l):
    try:
        if l.split(' ')[-2] == '404':
            return True
    except:
        return False

Error404July = rddJuly.filter(check_404).map(lambda x: (x.split( )[6], x.split( )[-2])).filter(lambda line: '404' in line).map(lambda x: (x,1)).reduceByKey(lambda k,v : k+v).map(lambda x: (x[1], x[0])).sortByKey(ascending=False)

print('***** TOP FIVE URL ERRORS 404 JULY *****')
top = dict(Error404July.take(7))
x = 0
for i in top:
    l = list(top.values())[x]
    print('TOP:', x + 1, '- URL:', l[0] , ' - Errors:', list(top.keys())[x])
    x = x + 1

***** TOP FIVE URL ERRORS 404 JULY *****
TOP: 1 - URL: /pub/winvn/readme.txt  - Errors: 667
TOP: 2 - URL: /pub/winvn/release.txt  - Errors: 547
TOP: 3 - URL: /history/apollo/apollo-13.html  - Errors: 286
TOP: 4 - URL: /shuttle/resources/orbiters/atlantis.gif  - Errors: 230
TOP: 5 - URL: /://spacelink.msfc.nasa.gov  - Errors: 215


In [13]:
Error404August = rddAugust.map(lambda x: (x.split( )[6], x.split( )[-2])).filter(lambda line: '404' in line).map(lambda x: (x,1)).reduceByKey(lambda k,v : k+v).map(lambda x: (x[1], x[0])).sortByKey(ascending=False)

print('***** TOP FIVE URL ERRORS 404 AUGUST *****')
top = dict(Error404August.take(5))
x = 0
for i in top:
    l = list(top.values())[x]
    print('TOP:', x + 1, '- URL: ', l[0] , ' - Errors: ', list(top.keys())[x])
    x = x + 1

***** TOP FIVE URL ERRORS 404 AUGUST *****
TOP: 1 - URL:  /pub/winvn/readme.txt  - Errors:  1337
TOP: 2 - URL:  /pub/winvn/release.txt  - Errors:  1185
TOP: 3 - URL:  /shuttle/missions/STS-69/mission-STS-69.html  - Errors:  682
TOP: 4 - URL:  /images/nasa-logo.gif  - Errors:  319
TOP: 5 - URL:  /shuttle/missions/sts-68/ksc-upclose.gif  - Errors:  251


4. Quantidade de erros 404 por dia.

In [14]:
def check_404(l):
    try:
        if l.split(' ')[-2] == '404':
            return True
    except:
        return False

Error404JulyDay = rddJuly.filter(check_404).map(lambda x: (x.split( )[3].split('[')[1].split('/')[0], x.split( )[-2])).filter(lambda line: '404' in line).sortByKey().countByKey()

print('***** Errors 404 per Day (July) *****')
x = 0
for i in Error404JulyDay:
    print('Day:', list(Error404JulyDay.keys())[x], ' - Errors: ', list(Error404JulyDay.values())[x])
    x = x + 1


***** Errors 404 per Day (July) *****
Day: 01  - Errors:  316
Day: 02  - Errors:  291
Day: 03  - Errors:  474
Day: 04  - Errors:  359
Day: 05  - Errors:  497
Day: 06  - Errors:  640
Day: 07  - Errors:  570
Day: 08  - Errors:  302
Day: 09  - Errors:  348
Day: 10  - Errors:  398
Day: 11  - Errors:  471
Day: 12  - Errors:  471
Day: 13  - Errors:  532
Day: 14  - Errors:  413
Day: 15  - Errors:  254
Day: 16  - Errors:  257
Day: 17  - Errors:  406
Day: 18  - Errors:  465
Day: 19  - Errors:  639
Day: 20  - Errors:  428
Day: 21  - Errors:  334
Day: 22  - Errors:  192
Day: 23  - Errors:  233
Day: 24  - Errors:  328
Day: 25  - Errors:  461
Day: 26  - Errors:  336
Day: 27  - Errors:  336
Day: 28  - Errors:  94


In [15]:
Error404AugustDay = rddAugust.map(lambda x: (x.split( )[3].split('[')[1].split('/')[0], x.split( )[-2])).filter(lambda line: '404' in line).sortByKey().countByKey()

print('***** Errors 404 per Day (Agosto) *****')
x = 0
for i in Error404AugustDay:
    print('Day:', list(Error404AugustDay.keys())[x], ' - Errors: ', list(Error404AugustDay.values())[x])
    x = x + 1


***** Errors 404 per Day (Agosto) *****
Day: 01  - Errors:  243
Day: 03  - Errors:  304
Day: 04  - Errors:  346
Day: 05  - Errors:  236
Day: 06  - Errors:  373
Day: 07  - Errors:  537
Day: 08  - Errors:  391
Day: 09  - Errors:  279
Day: 10  - Errors:  315
Day: 11  - Errors:  263
Day: 12  - Errors:  196
Day: 13  - Errors:  216
Day: 14  - Errors:  287
Day: 15  - Errors:  327
Day: 16  - Errors:  259
Day: 17  - Errors:  271
Day: 18  - Errors:  256
Day: 19  - Errors:  209
Day: 20  - Errors:  312
Day: 21  - Errors:  305
Day: 22  - Errors:  288
Day: 23  - Errors:  345
Day: 24  - Errors:  420
Day: 25  - Errors:  415
Day: 26  - Errors:  366
Day: 27  - Errors:  370
Day: 28  - Errors:  410
Day: 29  - Errors:  420
Day: 30  - Errors:  571
Day: 31  - Errors:  526


5. O total de bytes retornados. 


In [16]:
JulyBytesTotal = rddJuly.map(lambda x: (1,x.split( )[-1])).map(lambda x : x[0]).sum()

print('Total bytes returned in July: ', JulyBytesTotal)

Total bytes returned in July:  1891715


In [17]:
AugustBytesTotal = rddAugust.map(lambda x: (1,x.split( )[-1])).map(lambda x : x[0]).sum()

print('Total bytes returned in August: ', AugustBytesTotal)

Total bytes returned in August:  1569898
